In [23]:
import requests
import json
import pandas as pd

In [27]:
def get_top_github_projects(x, token=None):
    headers = {'Authorization': f'token {token}'} if token else {}
    url = 'https://api.github.com/search/repositories'
    params = {
        'q': 'topic:javascript',
        'sort': 'stars',
        'order': 'desc',
        'per_page': x,  # number of top repositories
        'page': 1
    }

    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        pretty_json = json.dumps(data['items'], indent=4)
        # print(pretty_json)
        projects = [{'id': item['id'], 'name': item['name'], 'watchers': item['watchers_count'],
                     'created_at': item['created_at'],'updated_at': item['updated_at']
                     ,'size': item['size'],'stars': item['stargazers_count']
                     ,'open_issues_count': item['open_issues_count']
                     ,'forks': item['forks']} for item in data['items']]
        df = pd.DataFrame(projects)
        return df

    else:
        print(f"Error: {response.status_code}")
        return None

top_projects = get_top_github_projects(10)
top_projects.head()

,id,name,watchers,created_at,updated_at,size,stars,open_issues_count,forks
0,28457823,freeCodeCamp,404692,2014-12-24T17:49:19Z,2024-10-29T05:44:39Z,471186,404692,247,37947
1,10270250,react,228763,2013-05-24T16:15:54Z,2024-10-29T06:46:14Z,633039,228763,860,46824
2,11730342,vue,207903,2013-07-29T03:24:51Z,2024-10-29T06:21:19Z,32152,207903,603,33688
3,88011908,project-based-learning,202366,2017-04-12T05:07:46Z,2024-10-29T07:06:09Z,438,202366,188,26406
4,126577260,javascript-algorithms,188163,2018-03-24T07:47:04Z,2024-10-29T05:35:47Z,13398,188163,352,30237
